In [2]:
# Permet de tout executer au lancement du notebook + conserver le notebook actif pendant 2h
from IPython.display import Javascript
import sys
if ".." not in sys.path:
    sys.path.append("..")
from masquer import *
Javascript("""
function repeter(){
IPython.notebook.kernel.execute("a=1");
}
// execute a = 1 en python toutes les 8 minutes pendant 2h
let timerId = setInterval(() => repeter(), 4800);
setTimeout(() => { clearInterval(timerId); alert('fin de cession'); }, 7200000);

// Supprimer la taille limite pour la sortie d'une cellule
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
};
IPython.notebook.kernel.execute("url = '" + window.location + "'");

// Exécuter toutes les cellule du notebook
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            
                
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
                Jupyter.actions.call('jupyter-notebook:hide-header')

        }
    );""")

<IPython.core.display.Javascript object>

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
HTML("""<style>
h1 {
  font-family: 'Permanent Marker', cursive;
  text-align: center;
  color: red;
  
}
ol {
  list-style-position: inside;
  margin-left: 1em;
  list-style-position: outside;
}
h2 {
  font-family: 'Permanent Marker', cursive;
  color: blue;
}
h3 {
  font-family: 'Permanent Marker', cursive;

}
</style>""")

# TD_08_3 - Le langage SQL 2/2

On peut aussi utiliser la base de données présente localement en utilisant la bibliothèque **sqlite3** de python. La requête SQL est alors envoyée sous forme d'une chaine de caractères

In [6]:
import sqlite3
connection = sqlite3.connect("ipt.sqlite3")
cursor = connection.cursor()

La base **ipt.sqlite3** contient 6 tables dont la table **album**

In [7]:
# Affichage des différentes relations disponibles
for row in cursor.execute("""SELECT name FROM sqlite_master WHERE type='table';"""):
    print(row)

('album',)
('bbc',)
('cia',)
('style',)
('totp',)
('track',)


In [8]:
for row in cursor.execute("PRAGMA table_info(album);"):
    print(row)

(0, 'asin', 'char(10)', 1, None, 0)
(1, 'title', 'varchar(50)', 0, 'NULL', 0)
(2, 'artist', 'varchar(50)', 0, 'NULL', 0)
(3, 'price', 'decimal(10,2)', 0, 'NULL', 0)
(4, 'rdate', 'date', 0, 'NULL', 0)
(5, 'label', 'varchar(50)', 0, 'NULL', 0)
(6, 'rank', 'integer', 0, 'NULL', 0)


Les donnees sont issues d’amazon. L’attribut **price** représente le prix en dollars, l’attribut **rank** est le rang dans les ventes de disques. L'**ASIN** (Amazon Standard Identification Number) est un bloc unique de 10 lettres et/ou chiffres qui identifient les articles. L'ASIN figure sur la page d'informations du produit sur le site Amazon.fr.

## Exercice 1
1. Chercher tous les albums de U2
2. Chercher tous les albums parus en 1991

On peut utiliser la requête agrégative **COUNT** et le mot clé **GROUP BY** pour indiquer quelles lignes doivent être regroupées. Il existe aussi **AVG** (average), **MAX**, **MIN** et **SUM**. Pour filtrer selon une condition après avoir regroupé les lignes, on utilise le mot clé **HAVING**, le mot clé **WHERE** est utilisé pour filtrer les lignes avant les avoir regroupées.

## Exercice 2
3. Qui a le plus d’albums dans les 10000 meilleures ventes le label Sony ou le label Atlantic ?
4. Combien d’artistes ont un album dans les 5000 meilleures ventes ?

On considérera aussi la table **track**

In [9]:
for row in cursor.execute("PRAGMA table_info(track);"):
    print(row)

(0, 'album', 'char(10)', 1, None, 0)
(1, 'disk', 'integer', 1, None, 0)
(2, 'posn', 'integer', 1, None, 0)
(3, 'song', 'varchar(255)', 0, 'NULL', 0)


L’attribut **album** de cette table reprend l’attribut **asin** de la table précédente pour identifier chaque album.

Il est possible de filtrer en utilisant un masque à l'aide du mot clé **LIKE**. Le masque peut contenir les caractères % qui représente n'importe quelle chaine de caractère (y compris une chaine vide) et _ qui représente n'importe quel caractère (un seul). WHERE name LIKE '%e%e%' va conserver tous les noms contenant 2 e par exemple.

## Exercice 3
5. Chercher tous les asin des albums parus en 1991. En déduire la liste de tous les titres de chansons dans des albums parus en 1991.
6. Chercher l’ensemble des **song**, **title**, **artist** pour les chansons commençant par S d’artistes commençant par B 


In [10]:
connection = sqlite3.connect("ipt3.sqlite3")
cursor = connection.cursor()

La base **ipt3.sqlite3** contient 3 tables:
* actor
* casting
* movie

In [11]:
# Affichage des différentes relations disponibles
for row in cursor.execute("""SELECT name FROM sqlite_master WHERE type='table';"""):
    print(row)

('actor',)
('casting',)
('movie',)


In [12]:
for row in cursor.execute("PRAGMA table_info(actor);"):
    print(row)

(0, 'id', 'integer', 1, None, 0)
(1, 'name', 'varchar(35)', 0, 'NULL', 0)


Dans la table **actor**, l'attribut **id** permet d'identifier un acteur de manière unique (clé primaire) l'associe à son nom **name**. Cette table contient également les réalisateurs.

In [13]:
for row in cursor.execute("PRAGMA table_info(movie);"):
    print(row)

(0, 'id', 'integer', 1, None, 0)
(1, 'title', 'varchar(70)', 0, 'NULL', 0)
(2, 'yr', 'decimal(4,0)', 0, 'NULL', 0)
(3, 'score', 'float', 0, 'NULL', 0)
(4, 'votes', 'integer', 0, 'NULL', 0)
(5, 'director', 'integer', 0, 'NULL', 0)


La table **movie** contient un attribut **id** (clé primaire) permettant d'identifier de manière unique chaque ligne. Cet id est associé au titre du film **title**, à l'année **yr** sous forme d'un nombre à 4 chiffres, à son réalisator **director**. L'attribut director fait référence à un id de la table actor. Un pur réalisateur peut ne pas être un vrai acteur mais sera présent dans la table actor quand même.


In [14]:
for row in cursor.execute("PRAGMA table_info(casting);"):
    print(row)

(0, 'movieid', 'integer', 1, None, 0)
(1, 'actorid', 'integer', 1, None, 0)
(2, 'ord', 'integer', 0, 'NULL', 0)


La table casting permet de mettre en relation la table actor et la table movie. L'entier **ord** indique l'ordre dans lequel sont énumérés les acteurs dans le casting d'un film : du premier nom de valeur 1 (le plus important) au dernier.

## Exercice 4
7. Déterminez le nom de tous les films qui commencent par « star » (16 réponses)
8. Déterminez le nombre d’acteurs pour chacun de ces films (sans les compter à la main:) )
9. Déterminez le nombre maximum d’acteurs au casting dans un film de cette base.
10. Déterminez les identifiants d’acteurs qui n’ont joué qu’en étant premier au casting
11. Déterminez pour chacun d’eux combien de films ils ont tourné
12. Déterminez le nom de tous les acteurs n’ayant joué qu’en étant premier au casting et ayant tourné plus de 5 films.
13. Ecrire une requete permettant d’afficher la liste des noms des réalisateurs (sans doublon)
14. Déterminer les noms des acteurs qui n’ont tourné qu’avec un seul réalisateur.
15. Déterminer les noms des acteurs qui n’ont tourné que dans des films qu’ils ont réalisés.